### Análise da proporção de mensagens de falha por tipo de voo

Este notebook retrata a análise da proporção de incidência de falhas por tipo de voo. Nesse caso, a divisão de tipo de voo é feita da seguinte forma: parte dos arquivos de voo retratam voos sem inconsistências aparentes (isto é, apresentam as três fases de voo que uma aeronave realiza), enquanto os arquivos restantes apresentam uma inconsistência identificada (uma única fase de voo é registrada no arquivo).

In [9]:
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [10]:
df_falha = None
df_normal = None
folder = "./data"
archives = [arq for arq in os.listdir(folder) if arq.endswith('.parquet')]
columns = ['message0418DAA-1', 'message0422DAA-1', 'phaseOfFlight-1']

In [11]:
for archive in archives:
    voo_normal = True
    path_archive = os.path.join(folder, archive)
    df_temp = pd.read_parquet(path_archive, columns=columns)
    df_temp['flight'] = archive
    df_temp['message0418DAA-1'] = df_temp['message0418DAA-1'].astype('Int8')
    df_temp['message0422DAA-1'] = df_temp['message0422DAA-1'].astype('Int8')
    df_temp['phaseOfFlight-1'] = df_temp['phaseOfFlight-1'].astype('Int8')
    df_temp.fillna(method='ffill', inplace=True)
    df_temp.fillna(method='bfill', inplace=True)
    estados_voo = df_temp['phaseOfFlight-1'].unique()
    if len(estados_voo) == 1:
        if df_falha is None:
            df_falha = df_temp
        else:
            df_falha = pd.concat([df_falha, df_temp])
    df_temp = None
    estados_voo = None

In [12]:
contador = 0
for archive in archives:
    voo_normal = True
    path_archive = os.path.join(folder, archive)
    df_temp = pd.read_parquet(path_archive, columns=columns)
    df_temp['flight'] = archive
    df_temp['message0418DAA-1'] = df_temp['message0418DAA-1'].astype('Int8')
    df_temp['message0422DAA-1'] = df_temp['message0422DAA-1'].astype('Int8')
    df_temp['phaseOfFlight-1'] = df_temp['phaseOfFlight-1'].astype('Int8')
    df_temp.fillna(method='ffill', inplace=True)
    df_temp.fillna(method='bfill', inplace=True)

    if df_normal is None:
        df_normal = df_temp
    else:
        df_normal = pd.concat([df_normal, df_temp])
    df_temp = None
    contador += 1
    if contador % 163 == 0:
        break

In [13]:
df_falha.fillna(method='ffill', inplace=True)
df_falha.fillna(method='bfill', inplace=True)
df_normal.fillna(method='ffill', inplace=True)
df_normal.fillna(method='bfill', inplace=True)

In [19]:
print(df_normal.shape)
print(df_falha.shape)

(19572379, 4)
(2833869, 4)


In [25]:
df_falha["flight"].nunique()
df_normal["flight"].nunique()

163

In [26]:
df_falha

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
0,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
1,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
2,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
3,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
4,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
...,...,...,...,...
14536,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14537,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14538,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14539,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet


In [57]:
# Salvar a quantidade de 5 na coluna
try:
    quantidade_erros_normal_418 = df_normal['message0418DAA-1'].value_counts()[5]
except:
    quantidade_erros_normal_418 = 0

try:
    quantidade_erros_normal_422 = df_normal['message0422DAA-1'].value_counts()[5]
except:
    quantidade_erros_normal_422 = 0
    
try:
    quantidade_erros_falha_418 = df_falha['message0418DAA-1'].value_counts()[5]
except:
    quantidade_erros_falha_418 = 0

try:
    quantidade_erros_falha_422 = df_falha['message0422DAA-1'].value_counts()[5]
except:
    quantidade_erros_falha_422 = 0

quantidade_erros_normal_total = quantidade_erros_normal_418 + quantidade_erros_normal_422
quantidade_erros_falha_total = quantidade_erros_falha_418 + quantidade_erros_falha_422
print(quantidade_erros_normal_total)
print(quantidade_erros_falha_total)

# quantidade_erros_falha = int(df_falha['message0418DAA-1'].value_counts()[5]) + int(df_falha['message0422DAA-1'].value_counts()[5])
# print(quantidade_erros_normal)
# print(quantidade_erros_falha)

200
260


In [58]:
#obter número de linhas do df 
horas_normal = df_normal.shape[0]/72000
horas_falha = df_falha.shape[0]/72000
print(horas_normal)
print(horas_falha)

271.83859722222223
39.359291666666664


In [60]:
print(quantidade_erros_falha_total/horas_falha)
print(quantidade_erros_normal_total/horas_normal)

6.605809936874288
0.7357306947714429
